In [3]:
import shadie
import numpy as np
import pyslim
import toyplot
import tskit
import toytree
import os
import scipy

#to suppress pandas FutureWarning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd

In [4]:
from toytree.utils.src.style_axes import set_axes_ticks_external as set_axes_ticks_external
from toytree.utils.src.style_axes import set_axes_box_outline as set_axes_box_outline
import toyplot.svg

## Chromosome Set-Up

In [5]:
#default exon and intron settings from shadie
e1 = shadie.EXON 
e2 = shadie.INTRON
e3 = shadie.NONCDS

In [6]:
neut_chrom = shadie.chromosome.explicit(data = {(0, 31560): e3},
                                   use_synonymous_sites_in_coding=False,
                                   )
neut_chrom.draw();

(<toyplot.canvas.Canvas at 0x15551fd05510>,
 <toyplot.mark.FillBoundaries at 0x15551f29e810>)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="700.0px" height="150.0px" viewBox="0 0 700.0 150.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tbab5026ca3244cfa944f8043db7e13d2"> name: noncds
interval: (0, 31560)
ElementType: g3
is coding: False 0 5000 10000 15000 20000 25000 30000

In [7]:
del_chrom = shadie.chromosome.explicit(data = {(0, 9999): e3,
                                                  (10000,22279): e2,
                                                  (22280, 31560): e3
                                                 },
                                   use_synonymous_sites_in_coding=False,
                                   )
del_chrom.draw();

(<toyplot.canvas.Canvas at 0x155551462e50>,
 <toyplot.mark.FillBoundaries at 0x15551f2be990>)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="700.0px" height="150.0px" viewBox="0 0 700.0 150.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t410b51bb9ff3424286a422870a818cbd"> name: noncds
interval: (0, 9999)
ElementType: g3
is coding: False name: intron
interval: (10000, 22279)
ElementType: g2
is coding: True name: noncds
interval: (22280, 31560)
ElementType: g3
is coding: False 0 5000 10000 15000 20000 25000 30000

In [8]:
ben_chrom = shadie.chromosome.explicit(data = {(0, 9999): e3,
                                                  (10000, 10359): e1,
                                                  (10360, 15959): e2,
                                                  (15960, 16319): e1,
                                                  (16320, 21919): e2,
                                                  (21920, 22279): e1,
                                                  (22280, 31560): e3
                                                 },
                                   use_synonymous_sites_in_coding=False,
                                   )
ben_chrom.draw();

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="700.0px" height="150.0px" viewBox="0 0 700.0 150.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tfd07b1a3ec264ffdaeca87c6733f8cca"> name: noncds
interval: (0, 9999)
ElementType: g3
is coding: False name: exon
interval: (10000, 10359)
ElementType: g1
is coding: True name: intron
interval: (10360, 15959)
ElementType: g2
is coding: True name: exon
interval: (15960, 16319)
ElementType: g1
is coding: True name: intron
interval: (16320, 21919)
ElementType: g2
is coding: True name: exon
interval: (21920, 22279)
ElementType: g1
is coding: True name: noncds
interval: (22280, 31560)
ElementType: g3
is coding: False 0 5000 10000 15000 20000 25000 30000

## Saving Data from `.tree` file to `.csv`
The `save_data_het()` function saves the heterozygosity values calculated during the SLiM simulation to a `.csv` file and also calculates confidence intervals, standard deviation, Ne, and Fis. If you run your own simulations, you only need to do this step once, then you can work from the `.csv` files for all future analyses. 
The treesequence files for this study were too large to upload to GitHub, so we provided the `.csv` files in the `data` directory. If you are using that data, you can go ahead and skip down to the next section, "Loading the Data". 

In [11]:
def save_data_het(dirpath, mut_rate, chrom, model, outpath="git_data/"):
    
    ex_df = pd.DataFrame(columns = ['mut_rate', 'chrom', 'time', "mean_heterozygosity", "CI_5%", "CI_95%", "stdev"])
    ob_df = pd.DataFrame(columns = ['mut_rate', 'chrom', 'time', "mean_heterozygosity", "CI_5%", "CI_95%", "stdev"])
    theta_df = pd.DataFrame(columns = ['mut_rate', 'chrom', 'time', "mean_heterozygosity", "CI_5%", "CI_95%", 
                                       "stdev", ])
    
    #initialize the dictionary
    directory = dirpath
    
    #save the sampled times to a list
    times = []
    
    #for vittaria
    if model=="vittaria":
        for i in range(1,201):
            time = i*200
            times.append(time)

    #for normal ferns
    else:
        for i in range(1,200):
            time = i*200
            times.append(time+1)
    
    final_ex_dict = {int(x): [] for x in times}
    final_ob_dict = {int(x): [] for x in times}
    final_theta_dict = {int(x): [] for x in times}
    
    #add data to the dictionary
    for filename in os.listdir(directory):
        #generate the full path
        f = os.path.join(directory, filename)
        
        # check if it is a file
        if os.path.isfile(f):
            #load the file
            ts = tskit.load(f)
            
            het_dict = ts.metadata['SLiM']['user_metadata']['ex_heterozygosity'][0]
            int_dict = {int(k):v for k,v in het_dict.items()}
            sorted_dict = {key:int_dict[key] for key in sorted(int_dict)}
            
            for key, value in sorted_dict.items():
                final_ex_dict[key].append(float(1000000*value[0]))
                
            ob_dict = ts.metadata['SLiM']['user_metadata']['ob_heterozygosity'][0]
            ob_int_dict = {int(k):v for k,v in ob_dict.items()}
            ob_sorted_dict = {key:ob_int_dict[key] for key in sorted(ob_int_dict)}
            
            for key, value in ob_sorted_dict.items():
                final_ob_dict[key].append(float(1000000*value[0]))
                
            theta_dict = ts.metadata['SLiM']['user_metadata']['theta'][0]
            theta_int_dict = {int(k):v for k,v in theta_dict.items()}
            theta_sorted_dict = {key:theta_int_dict[key] for key in sorted(theta_int_dict)}
            
            for key, value in theta_sorted_dict.items():
                final_theta_dict[key].append(float(1000000*value[0]))

    for key, value in final_ex_dict.items():
        mean_val = np.mean(value)
        low, high = scipy.stats.t.interval(confidence=0.95,
            df=len(value)-1,
            loc=mean_val,
            scale=scipy.stats.sem(value),
        )
        std = np.std(value)
    
        ex_df = pd.concat([pd.DataFrame([[mut_rate, chrom, key, mean_val, low, high, std]], columns=ex_df.columns), ex_df], 
                   ignore_index=True)
    
    #save the df
    ex_file = outpath + f"{model}_" + f"{chrom}_" + f"{mut_rate}_" + "ex_heterozygosity.csv"
    
    ex_df.to_csv(ex_file, header = True, index = False)
    
    ex_vals = ex_df["mean_heterozygosity"]
    
    for key, value in final_ob_dict.items():
        mean_val = np.mean(value)
        low, high = scipy.stats.t.interval(
            confidence=0.95, 
            df=len(value)-1, 
            loc=mean_val, 
            scale=scipy.stats.sem(value),
        )
        std = np.std(value)
    
        ob_df = pd.concat([pd.DataFrame([[mut_rate, chrom, key, mean_val, low, high, std]], columns=ob_df.columns), ob_df], 
                   ignore_index=True)
    
    #save the df
    ob_file = outpath + f"{model}_" + f"{chrom}_" + f"{mut_rate}_" + "ob_heterozygosity.csv"
    
    ob_df.to_csv(ob_file, header = True, index = False)
    
    ob_vals = ob_df["mean_heterozygosity"]
    
    for key, value in final_theta_dict.items():
        mean_val = np.mean(value)
        low, high = scipy.stats.t.interval(
            confidence=0.95, 
            df=len(value)-1, 
            loc=mean_val,
            scale=scipy.stats.sem(value),
        )
        std = np.std(value)
    
        theta_df = pd.concat([pd.DataFrame([[mut_rate, chrom, key, mean_val, low, high, std]], 
                                           columns=theta_df.columns), theta_df], ignore_index=True)
    
    #add Ne and Fis to theta file
    
    # define SLiM mutation rate
    if mut_rate == "low":  
        MUT = 5e-10/2
    elif mut_rate == "med":
        MUT = 5e-9/2
    elif mut_rate == "high":
        MUT = 5e-8/2
    
    Ne = (1e-6*theta_df["mean_heterozygosity"])/(MUT*4)
    Fis = (ex_vals-ob_vals)/ex_vals
    
    theta_df["Ne"] = Ne 
    theta_df["Fis"] = Fis
    
    #save the df
    theta_file = outpath + f"{model}_" + f"{chrom}_" + f"{mut_rate}_" + "theta.csv"
    
    theta_df.to_csv(theta_file, header = True, index = False)

### Neutral *Vittaria* sims

In [8]:
save_data_het(dirpath="../jan2024/neutral/lowmut/trees/", mut_rate="low", chrom="neut", model="vittaria")

In [9]:
save_data_het(dirpath="../jan2024/neutral/medmut/trees/", mut_rate="med", chrom="neut", model="vittaria")

In [12]:
save_data_het(dirpath="../jan2024/neutral/highmut/trees/", mut_rate="high", chrom="neut", model="vittaria")

### Purifying *Vittaria* sims

In [13]:
save_data_het(dirpath="purifying/lowmut/trees/", mut_rate="low", chrom="pur", model="vittaria")

In [14]:
save_data_het(dirpath="purifying/medmut/trees/", mut_rate="med", chrom="pur", model="vittaria")

In [15]:
save_data_het(dirpath="purifying/highmut/trees/", mut_rate="high", chrom="pur", model="vittaria")

### Postive *Vittaria* sims

In [16]:
save_data_het(dirpath="positive/lowmut/trees/", mut_rate="low", chrom="ben", model="vittaria")

In [17]:
save_data_het(dirpath="positive/medmut/trees/", mut_rate="med", chrom="ben", model="vittaria")

In [18]:
save_data_het(dirpath="positive/highmut/trees/", mut_rate="high", chrom="ben", model="vittaria")

### *Vittaria* sims with Sexual Selection

In [24]:
save_data_het(dirpath="sexual/lowsex/positive/trees/", mut_rate="med", chrom="pos-sex-low", model="vittaria")

In [25]:
save_data_het(dirpath="sexual/lowsex/purifying/trees/", mut_rate="med", chrom="pur-sex-low", model="vittaria")

In [26]:
save_data_het(dirpath="../jan2024/sexual/neutral/trees/", mut_rate="med", chrom="neut-sex-low", model="vittaria")

In [27]:
save_data_het(dirpath="sexual/medsex/positive/trees/", mut_rate="med", chrom="pos-sex-med", model="vittaria")

In [28]:
save_data_het(dirpath="sexual/medsex/purifying/trees/", mut_rate="med", chrom="pur-sex-med", model="vittaria")

In [29]:
save_data_het(dirpath="../jan2024/sexual/medsex/neutral/trees/", mut_rate="med", chrom="neut-sex-med", model="vittaria")

In [30]:
save_data_het(dirpath="sexual/highsex/positive/trees/", mut_rate="med", chrom="pos-sex-high", model="vittaria")

In [31]:
save_data_het(dirpath="sexual/highsex/purifying/trees/", mut_rate="med", chrom="pur-sex-high", model="vittaria")

In [32]:
save_data_het(dirpath="../jan2024/sexual/highsex/neutral/trees/", mut_rate="med", chrom="neut-sex-high", model="vittaria")

### Homosporous Fern sims

In [36]:
save_data_het(dirpath="homosporous/neutral/lowmut/trees/", mut_rate="low", chrom="neut", model="homfern")

In [37]:
save_data_het(dirpath="homosporous/purifying/lowmut/trees/", mut_rate="low", chrom="del", model="homfern")

In [38]:
save_data_het(dirpath="homosporous/positive/lowmut/trees/", mut_rate="low", chrom="ben", model="homfern")

In [33]:
save_data_het(dirpath="homosporous/neutral/medmut/trees/", mut_rate="med", chrom="neut", model="homfern")

In [34]:
save_data_het(dirpath="homosporous/purifying/medmut/trees/", mut_rate="med", chrom="del", model="homfern")

In [35]:
save_data_het(dirpath="homosporous/positive/medmut/trees/", mut_rate="med", chrom="ben", model="homfern")

## Loading the Data
Once you save the data as a .csv using the function above, you can skip to this step and simply load the data from your `.csv` files. 

### Neutral Sims

In [42]:
neut_low_ex = pd.read_csv("git_data/vittaria_neut_low_ex_heterozygosity.csv")
neut_low_ob = pd.read_csv("git_data/vittaria_neut_low_ob_heterozygosity.csv")
neut_low_theta = pd.read_csv("git_data/vittaria_neut_low_theta.csv")

In [43]:
hom_neut_low_ex = pd.read_csv("git_data/homfern_neut_low_ex_heterozygosity.csv")
hom_neut_low_ob = pd.read_csv("git_data/homfern_neut_low_ob_heterozygosity.csv")
hom_neut_low_theta = pd.read_csv("git_data/homfern_neut_low_theta.csv")

In [44]:
neut_med_ex = pd.read_csv("git_data/vittaria_neut_med_ex_heterozygosity.csv")
neut_med_ob = pd.read_csv("git_data/vittaria_neut_med_ob_heterozygosity.csv")
neut_med_theta = pd.read_csv("git_data/vittaria_neut_med_theta.csv")

In [45]:
hom_neut_med_ex = pd.read_csv("git_data/homfern_neut_med_ex_heterozygosity.csv")
hom_neut_med_ob = pd.read_csv("git_data/homfern_neut_med_ob_heterozygosity.csv")
hom_neut_med_theta = pd.read_csv("git_data/homfern_neut_med_theta.csv")

In [46]:
neut_high_ex = pd.read_csv("git_data/vittaria_neut_high_ex_heterozygosity.csv")
neut_high_ob = pd.read_csv("git_data/vittaria_neut_high_ob_heterozygosity.csv")
neut_high_theta = pd.read_csv("git_data/vittaria_neut_high_theta.csv")

### Purifying Selection Sims
#### Low mutation rate

In [47]:
pur_low_ex = pd.read_csv("git_data/vittaria_pur_low_ex_heterozygosity.csv")
pur_low_ob = pd.read_csv("git_data/vittaria_pur_low_ob_heterozygosity.csv")
pur_low_theta = pd.read_csv("git_data/vittaria_pur_low_theta.csv")

In [48]:
hom_pur_low_ex = pd.read_csv("git_data/homfern_del_low_ex_heterozygosity.csv")
hom_pur_low_ob = pd.read_csv("git_data/homfern_del_low_ob_heterozygosity.csv")
hom_pur_low_theta = pd.read_csv("git_data/homfern_del_low_theta.csv")

#### Medium mutation rate

In [49]:
pur_med_ex = pd.read_csv("git_data/vittaria_pur_med_ex_heterozygosity.csv")
pur_med_ob = pd.read_csv("git_data/vittaria_pur_med_ob_heterozygosity.csv")
pur_med_theta = pd.read_csv("git_data/vittaria_pur_med_theta.csv")

In [50]:
hom_pur_med_ex = pd.read_csv("git_data/homfern_del_med_ex_heterozygosity.csv")
hom_pur_med_ob = pd.read_csv("git_data/homfern_del_med_ob_heterozygosity.csv")
hom_pur_med_theta = pd.read_csv("git_data/homfern_neut_med_theta.csv")

#### High mutation rate

In [53]:
pur_high_ex = pd.read_csv("git_data/vittaria_pur_high_ex_heterozygosity.csv")
pur_high_ob = pd.read_csv("git_data/vittaria_pur_high_ob_heterozygosity.csv")
pur_high_theta = pd.read_csv("git_data/vittaria_pur_high_theta.csv")

### Positive Selection Sims
#### Low mutation rate

In [51]:
pos_low_ex = pd.read_csv("git_data/vittaria_ben_low_ex_heterozygosity.csv")
pos_low_ob = pd.read_csv("git_data/vittaria_ben_low_ob_heterozygosity.csv")
pos_low_theta = pd.read_csv("git_data/vittaria_ben_low_theta.csv")

In [52]:
hom_pos_low_ex = pd.read_csv("git_data/homfern_ben_low_ex_heterozygosity.csv")
hom_pos_low_ob = pd.read_csv("git_data/homfern_ben_low_ob_heterozygosity.csv")
hom_pos_low_theta = pd.read_csv("git_data/homfern_ben_low_theta.csv")

#### Medium mutation rate

In [54]:
pos_med_ex = pd.read_csv("git_data/vittaria_ben_med_ex_heterozygosity.csv")
pos_med_ob = pd.read_csv("git_data/vittaria_ben_med_ob_heterozygosity.csv")
pos_med_theta = pd.read_csv("git_data/vittaria_ben_med_theta.csv")

In [55]:
hom_pos_med_ex = pd.read_csv("git_data/homfern_ben_med_ex_heterozygosity.csv")
hom_pos_med_ob = pd.read_csv("git_data/homfern_ben_med_ob_heterozygosity.csv")
hom_pos_med_theta = pd.read_csv("git_data/homfern_ben_med_theta.csv")

#### High mutation rate

In [56]:
pos_high_ex = pd.read_csv("git_data/vittaria_ben_high_ex_heterozygosity.csv")
pos_high_ob = pd.read_csv("git_data/vittaria_ben_high_ob_heterozygosity.csv")
pos_high_theta = pd.read_csv("git_data/vittaria_ben_high_theta.csv")

### Sexual Selection Sims
All simulations with sexual reproduction had a medium mutation rate
#### Low rate of sexual reproduction

In [58]:
lowsex_neut_med_ex = pd.read_csv("git_data/vittaria_neut-sex-low_med_ex_heterozygosity.csv")
lowsex_neut_med_ob = pd.read_csv("git_data/vittaria_neut-sex-low_med_ob_heterozygosity.csv")
lowsex_neut_med_theta = pd.read_csv("git_data/vittaria_neut-sex-low_med_theta.csv")

In [57]:
lowsex_pur_med_ex = pd.read_csv("git_data/vittaria_pur-sex-low_med_ex_heterozygosity.csv")
lowsex_pur_med_ob = pd.read_csv("git_data/vittaria_pur-sex-low_med_ob_heterozygosity.csv")
lowsex_pur_med_theta = pd.read_csv("git_data/vittaria_pur-sex-low_med_theta.csv")

In [59]:
lowsex_pos_med_ex = pd.read_csv("git_data/vittaria_pos-sex-low_med_ex_heterozygosity.csv")
lowsex_pos_med_ob = pd.read_csv("git_data/vittaria_pos-sex-low_med_ob_heterozygosity.csv")
lowsex_pos_med_theta = pd.read_csv("git_data/vittaria_pos-sex-low_med_theta.csv")

#### Medium rate of sexual reproduction

In [60]:
medsex_neut_med_ex = pd.read_csv("git_data/vittaria_neut-sex-med_med_ex_heterozygosity.csv")
medsex_neut_med_ob = pd.read_csv("git_data/vittaria_neut-sex-med_med_ob_heterozygosity.csv")
medsex_neut_med_theta = pd.read_csv("git_data/vittaria_neut-sex-med_med_theta.csv")

In [61]:
medsex_pur_med_ex = pd.read_csv("git_data/vittaria_pur-sex-med_med_ex_heterozygosity.csv")
medsex_pur_med_ob = pd.read_csv("git_data/vittaria_pur-sex-med_med_ob_heterozygosity.csv")
medsex_pur_med_theta = pd.read_csv("git_data/vittaria_pur-sex-med_med_theta.csv")

In [63]:
medsex_pos_med_ex = pd.read_csv("git_data/vittaria_pos-sex-med_med_ex_heterozygosity.csv")
medsex_pos_med_ob = pd.read_csv("git_data/vittaria_pos-sex-med_med_ob_heterozygosity.csv")
medsex_pos_med_theta = pd.read_csv("git_data/vittaria_pos-sex-med_med_theta.csv")

#### High rate of sexual reproduction

In [67]:
highsex_neut_med_ex = pd.read_csv("git_data/vittaria_neut-sex-high_med_ex_heterozygosity.csv")
highsex_neut_med_ob = pd.read_csv("git_data/vittaria_neut-sex-high_med_ob_heterozygosity.csv")
highsex_neut_med_theta = pd.read_csv("git_data/vittaria_neut-sex-high_med_theta.csv")

In [65]:
highsex_pur_med_ex = pd.read_csv("git_data/vittaria_pur-sex-high_med_ex_heterozygosity.csv")
highsex_pur_med_ob = pd.read_csv("git_data/vittaria_pur-sex-high_med_ob_heterozygosity.csv")
highsex_pur_med_theta = pd.read_csv("git_data/vittaria_pur-sex-high_med_theta.csv")

In [68]:
highsex_pos_med_ex = pd.read_csv("git_data/vittaria_pos-sex-high_med_ex_heterozygosity.csv")
highsex_pos_med_ob = pd.read_csv("git_data/vittaria_pos-sex-high_med_ob_heterozygosity.csv")
highsex_pos_med_theta = pd.read_csv("git_data/vittaria_pos-sex-high_med_theta.csv")

In [64]:
#take a look at the data
medsex_neut_med_theta

,mut_rate,chrom,time,mean_heterozygosity,CI_5%,CI_95%,stdev,Ne,Fis
0,med,neut-sex-med,40000,21.704911,20.832203,22.577620,9.922402,2170.491139,-0.139410
1,med,neut-sex-med,39800,21.682611,20.772096,22.593127,10.352252,2168.261137,-0.130664
2,med,neut-sex-med,39600,21.514132,20.602955,22.425308,10.359770,2151.413168,-0.164941
3,med,neut-sex-med,39400,22.011861,21.099342,22.924381,10.375037,2201.186148,-0.156753
4,med,neut-sex-med,39200,21.442913,20.547964,22.337863,10.175272,2144.291308,-0.142510
...,...,...,...,...,...,...,...,...,...
195,med,neut-sex-med,1000,14.419531,13.750086,15.088976,7.611359,1441.953104,0.009867
196,med,neut-sex-med,800,13.988281,13.346938,14.629625,7.291859,1398.828115,0.017212
197,med,neut-sex-med,600,13.551470,12.956564,14.146377,6.763886,1355.147032,-0.007357
198,med,neut-sex-med,400,11.868085,11.279525,12.456645,6.691727,1186.808478,0.027571


In [69]:
hom_pos_med_ex

,mut_rate,chrom,time,mean_heterozygosity,CI_5%,CI_95%,stdev
0,med,ben,39801,7.015507,6.184482,7.846532,9.227069
1,med,ben,39601,6.965422,6.153841,7.777003,9.011179
2,med,ben,39401,6.987053,6.196304,7.777802,8.779876
3,med,ben,39201,6.942073,6.140240,7.743905,8.902937
4,med,ben,39001,6.892982,6.099050,7.686915,8.815219
...,...,...,...,...,...,...,...
194,med,ben,1001,2.099917,1.756426,2.443408,3.813860
195,med,ben,801,2.000004,1.674329,2.325680,3.616056
196,med,ben,601,1.685581,1.399202,1.971959,3.179729
197,med,ben,401,1.160698,0.947468,1.373927,2.367538


## Generate Table of Values at 10,000 gens
Our data contains values every 200 SLiM generations. The `get_data_at` function saves out a table of values at the chosen generation (parameter `time`, in SLiM generations)

In [84]:
def get_data_at(data_lists, control_lists, time=20000):
    mut_rates = []
    selections = []
    reg_times = []
    thetas = []
    Nes = []
    Fiss = []
    Hos = []
    Hes = []
    
    reg_time = int((time)/2)
    
    for item in data_lists:
        df = item[0]
        theta = round(df.loc[df['time'] == time, 'mean_heterozygosity'].item(),3)*0.000001
        Ne = int(df.loc[df['time'] == time, 'Ne'].item())
        Fis = round(df.loc[df['time'] == time, 'Fis'].item(),3)
        mut_rate = df.loc[df['time'] == time, 'mut_rate'].item()
        selection = df.loc[df['time'] == time, 'chrom'].item()
        
        df1 = item[1]
        Ho = round(df1.loc[df['time'] == time, 'mean_heterozygosity'].item(),3)*0.000001
        
        df2 = item[2]
        He = round(df2.loc[df['time'] == time, 'mean_heterozygosity'].item(),3)*0.000001
        
        mut_rates.append(mut_rate)
        selections.append(selection)
        reg_times.append(reg_time)
        thetas.append(theta)
        Nes.append(Ne)
        Fiss.append(Fis)
        Hos.append(Ho)
        Hes.append(He)
    
    ctime = time+1
    for item in control_lists:
        df = item[0]
        theta = round(df.loc[df['time'] == ctime, 'mean_heterozygosity'].item(),3)*0.000001
        Ne = int(df.loc[df['time'] == ctime, 'Ne'].item())
        Fis = round(df.loc[df['time'] == ctime, 'Fis'].item(),3)
        mut_rate = df.loc[df['time'] == ctime, 'mut_rate'].item()
        selection = df.loc[df['time'] == ctime, 'chrom'].item()
        
        df1 = item[1]
        Ho = round(df1.loc[df['time'] == ctime, 'mean_heterozygosity'].item(),3)*0.000001
        
        df2 = item[2]
        He = round(df2.loc[df['time'] == ctime, 'mean_heterozygosity'].item(),3)*0.000001
        
        mut_rates.append(mut_rate)
        selections.append(selection)
        reg_times.append(reg_time)
        thetas.append(theta)
        Nes.append(Ne)
        Fiss.append(Fis)
        Hos.append(Ho)
        Hes.append(He)
        
    list_of_tuples = list(zip(mut_rates, selections, reg_times, Hes, Hos, thetas, Fiss, Nes))
    data_at_time = pd.DataFrame(list_of_tuples, columns = ['Mutation Rate', 'Selection', 'Generation', 
                                                 'He', 'Ho', 'Theta', 'Fis', 'Ne'])
    data_at_time.to_csv(f"data_at_{reg_time}.csv")

In [86]:
# get data at 20,000 generations, which is equal to 10,000 full life cycles
get_data_at(data_lists_med, control_lists, time=20000)

## Preparing the Data
To make plotting easier, we save dataframes of interest to lists

In [70]:
data_lists_med = [[pur_med_theta, pur_med_ob, pur_med_ex], 
                  [pos_med_theta, pos_med_ob, pos_med_ex],
                  [neut_med_theta, neut_med_ob, neut_med_ex], 
                  [lowsex_pur_med_theta, lowsex_pur_med_ob, lowsex_pur_med_ex],
                  [lowsex_pos_med_theta, lowsex_pos_med_ob, lowsex_pos_med_ex],
                  [lowsex_neut_med_theta, lowsex_neut_med_ob, lowsex_neut_med_ex],
                  [medsex_pur_med_theta, medsex_pur_med_ob, medsex_pur_med_ex],
                  [medsex_pos_med_theta, medsex_pos_med_ob, medsex_pos_med_ex],
                  [medsex_neut_med_theta, medsex_neut_med_ob, medsex_neut_med_ex],
                  [highsex_neut_med_theta, highsex_neut_med_ob, highsex_neut_med_ex],
                  [highsex_pos_med_theta, highsex_pos_med_ob, highsex_pos_med_ex],
                  [highsex_pur_med_theta, highsex_pur_med_ob, highsex_pur_med_ex],
                 ]

In [71]:
control_lists = [[hom_pur_med_theta, hom_pur_med_ob, hom_pur_med_ex],
                      [hom_pos_med_theta, hom_pos_med_ob, hom_pos_med_ex],
                      [hom_neut_med_theta, hom_neut_med_ob, hom_neut_med_ex]]

In [72]:
data_list_theta = [pur_low_theta, pos_low_theta, neut_low_theta, pur_high_theta, pos_high_theta, neut_high_theta,
             pur_med_theta, pos_med_theta, neut_med_theta, lowsex_pur_med_theta, lowsex_pos_med_theta, 
             lowsex_neut_med_theta, medsex_pur_med_theta, medsex_pos_med_theta, medsex_neut_med_theta,
             highsex_neut_med_theta, highsex_pos_med_theta, highsex_pur_med_theta, hom_pur_med_theta, 
             hom_pos_med_theta, hom_neut_med_theta]

In [73]:
data_list_ob = [pur_low_ob, pos_low_ob, neut_low_ob, pur_high_ob, pos_high_ob, neut_high_ob,
             pur_med_ob, pos_med_ob, neut_med_ob, lowsex_pur_med_ob, lowsex_pos_med_ob, 
             lowsex_neut_med_ob, medsex_pur_med_ob, medsex_pos_med_ob, medsex_neut_med_ob,
             highsex_neut_med_ob, highsex_pos_med_ob, highsex_pur_med_ob, hom_pur_med_ob, 
             hom_pos_med_ob, hom_neut_med_ob]

In [74]:
data_list_ex = [pur_low_ex, pos_low_ex, neut_low_ex, pur_high_ex, pos_high_ex, neut_high_ex,
             pur_med_ex, pos_med_ex, neut_med_ex, lowsex_pur_med_ex, lowsex_pos_med_ex, 
             lowsex_neut_med_ex, medsex_pur_med_ex, medsex_pos_med_ex, medsex_neut_med_ex,
             highsex_neut_med_ex, highsex_pos_med_ex, highsex_pur_med_ex, hom_pur_med_ex, 
             hom_pos_med_ex, hom_neut_med_ex]

## Plotting the Data
Set up the values and color mappings

In [87]:
LOWSEX = .000001 #(one ind every 1000 gens))
MEDSEX = .00001 #(one ind every 100 gens))
HIGHSEX = .0001 #(one ind every 10 gens)

In [88]:
LOWMUT = 5e-10
MEDMUT = 5e-9
HIGHMUT = 5e-8

In [89]:
colormap = toyplot.color.brewer.map("Spectral", domain_min=0, domain_max=1)
colormap

In [90]:
low_med_high = colormap.colors([0.1, 0.62, 1.0])
low_med_high

array([(0.19607843, 0.53333333, 0.74117647, 1.),
       (0.99529412, 0.83921569, 0.51215686, 1.),
       (0.61960784, 0.00392157, 0.25882353, 1.)],
      dtype=[('r', '<f8'), ('g', '<f8'), ('b', '<f8'), ('a', '<f8')])

In [91]:
low = colormap.colors([0, 0.1, 0.15])
low

array([(0.36862745, 0.30980392, 0.63529412, 1.),
       (0.19607843, 0.53333333, 0.74117647, 1.),
       (0.29803922, 0.64705882, 0.69411765, 1.)],
      dtype=[('r', '<f8'), ('g', '<f8'), ('b', '<f8'), ('a', '<f8')])

In [95]:
med = colormap.colors([0.32, 0.41, 0.62])
med

array([(0.71686275, 0.8854902 , 0.63372549, 1.),
       (0.91176471, 0.96470588, 0.61137255, 1.),
       (0.99529412, 0.83921569, 0.51215686, 1.)],
      dtype=[('r', '<f8'), ('g', '<f8'), ('b', '<f8'), ('a', '<f8')])

In [96]:
high = colormap.colors([0.70, 0.82, 0.99])
high

array([(0.99215686, 0.68235294, 0.38039216, 1.),
       (0.93254902, 0.39058824, 0.27215686, 1.),
       (0.64117647, 0.02784314, 0.26392157, 1.)],
      dtype=[('r', '<f8'), ('g', '<f8'), ('b', '<f8'), ('a', '<f8')])

In [97]:
full = colormap.colors([0.0, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
full

array([(0.36862745, 0.30980392, 0.63529412, 1.),
       (0.28235294, 0.42156863, 0.68823529, 1.),
       (0.19607843, 0.53333333, 0.74117647, 1.),
       (0.4       , 0.76078431, 0.64705882, 1.),
       (0.67058824, 0.86666667, 0.64313725, 1.),
       (0.90196078, 0.96078431, 0.59607843, 1.),
       (1.        , 1.        , 0.74901961, 1.),
       (0.99607843, 0.87843137, 0.54509804, 1.),
       (0.99215686, 0.68235294, 0.38039216, 1.),
       (0.95686275, 0.42745098, 0.2627451 , 1.),
       (0.83529412, 0.24313725, 0.30980392, 1.),
       (0.61960784, 0.00392157, 0.25882353, 1.)],
      dtype=[('r', '<f8'), ('g', '<f8'), ('b', '<f8'), ('a', '<f8')])

In [98]:
full2 = colormap.colors([0.9, 0.00, 0.41, 0.03, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
full2

array([(0.83529412, 0.24313725, 0.30980392, 1.),
       (0.36862745, 0.30980392, 0.63529412, 1.),
       (0.91176471, 0.96470588, 0.61137255, 1.),
       (0.31686275, 0.37686275, 0.66705882, 1.),
       (0.28235294, 0.42156863, 0.68823529, 1.),
       (0.19607843, 0.53333333, 0.74117647, 1.),
       (0.4       , 0.76078431, 0.64705882, 1.),
       (0.67058824, 0.86666667, 0.64313725, 1.),
       (0.90196078, 0.96078431, 0.59607843, 1.),
       (1.        , 1.        , 0.74901961, 1.),
       (0.99607843, 0.87843137, 0.54509804, 1.),
       (0.99215686, 0.68235294, 0.38039216, 1.),
       (0.95686275, 0.42745098, 0.2627451 , 1.),
       (0.83529412, 0.24313725, 0.30980392, 1.),
       (0.61960784, 0.00392157, 0.25882353, 1.)],
      dtype=[('r', '<f8'), ('g', '<f8'), ('b', '<f8'), ('a', '<f8')])

### Function for Plotting Heterozygosity Data

In [99]:
def het_plot(datasets, error="CI", labels=[],
                        ymin = 0.0, 
                        ymax = 2000,
                        #xmin = 0, xmax = 0.3,
                        xlocations = [0, 4000, 8000, 12000, 16000, 20000],
                                     #24000, 28000, 32000, 36000, 40000], 
                        #ylocations=[0.0, 0.25, 0.50, 0.75, 0.9], #1.0
                        palette = "Set2",
                        plot_error = True,
                        title = "Heterozygosity",):
    xlabel = "Time (generations)"
    title = title
    
    #conf = int(100 - float(CIs[0][4:8]))

    #make the colors
    if isinstance(palette, str):
        palette = toyplot.color.brewer.palette(palette) #count=len(dataset)

    if error=="CI":
        error_str = "CI=95%"
    elif error=="std":
        error_str= "(σ)"
    
    label_style = {"text-anchor":"start", "-toyplot-anchor-shift":"5px", "font-size":"14px"}
    canvas = toyplot.Canvas(width=600, height=500)
    axes = canvas.cartesian(label = title, 
                            xlabel=xlabel, 
                            ylabel= f"Heterozygosity (\u00D710<sup>-6</sup>, {error_str})",
                            ymin = ymin, 
                            ymax = ymax,
                            #xmin = xmin, 
                            xmax = 20000,
                           )

    xlocations=xlocations
    #ylocations=ylocations
    axes.x.ticks.locator = toyplot.locator.Explicit(locations=xlocations) #format="{:.2f}"
    axes.x.ticks.labels.style = {"font-size":"13px"}
    axes.y.ticks.labels.style = {"font-size":"13px"}
    #axes.y.ticks.locator = toyplot.locator.Explicit(locations=ylocations)
    #axes.x.ticks.locator = toyplot.locator.Uniform(count=6)

    for x in range(len(datasets)):
        data = datasets[x]
        time = data['time'].tolist()

        mean_heterozygosity_str = data['mean_heterozygosity'].tolist()
        mean_heterozygosity=[float(i) for i in mean_heterozygosity_str]

        #format the error
        if error == "CI":
            for column in data.columns:
                if "CI_95%" in column:
                    error_plus = column
                if "CI_5%" in column:
                    error_minus = column
            CI_plus_str = data[error_plus].tolist()
            CI_plus=[float(i) for i in CI_plus_str]

            a = [mean_heterozygosity, CI_plus]
            y1 = list(map(sum,zip(*a)))

            CI_minus_str = data[error_minus].tolist()
            CI_minus=[float(i) for i in CI_minus_str]
            minus_CI = []
            for i in CI_minus:
                minus_CI.append(i*-1)

            b = [mean_heterozygosity, minus_CI]
            y2 = list(map(sum,zip(*b)))

        elif error == "std":
            prob_std_str = data['stdev'].tolist()
            prob_std=[float(i) for i in prob_std_str]

            a = [mean_heterozygosity, prob_std]
            y1 = list(map(sum,zip(*a)))

            minus_prob_std = []
            for i in prob_std:
                minus_prob_std.append(i*-1)

            b = [mean_heterozygosity, minus_prob_std]
            y2 = list(map(sum,zip(*b)))

        if plot_error == True:
            mark = axes.fill(time, y1, y2, opacity=0.2, color = palette[x])
        mark = axes.plot(time, mean_heterozygosity, color=palette[x])
        axes.text(16000, mean_heterozygosity[110], f"{labels[x]}", style=label_style, color = palette[x])

    #mark = axes.vlines(x=0.0)
    set_axes_ticks_external(axes)
    axes.label.style = {"font-size":"19px"}
    axes.x.label.style = {"font-size":"16px"}
    axes.y.label.style = {"font-size":"16px"}
    #set_axes_box_outline(axes)

    return canvas

### Heterozygosity Plots

In [107]:
vittaria = het_plot([neut_med_ex, pur_med_ex, pos_med_ex, lowsex_neut_med_ex, lowsex_pur_med_ex, lowsex_pos_med_ex, 
         medsex_neut_med_ex, medsex_pur_med_ex, medsex_pos_med_ex,
         highsex_neut_med_ex, highsex_pur_med_ex, highsex_pos_med_ex], 
         plot_error = False,
         error = "std",
         ymax = 150,
         labels = ["Neutral", "Purifying", "Positive", "Low Sex, Neutral", "Low Sex, Purifying",
                  "Low Sex Postiive", "Med Sex Neutral", "Med Sex, Purifying", "Med Sex, Positive",
                  "High Sex, Neutral", "High Sex, Purifying", "High Sex, Positive"],
        title = "Expected Heterozygosity",
        palette = full)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600.0px" height="500.0px" viewBox="0 0 600.0 500.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t3c0454516f4d4d6d9374b83f8d70405f"> <path d="M 1020.0 175.6745523669358 L 1015.1999999999999 177.52840505549463 L 1010.4000000000001 179.44858589290936 L 1005.5999999999999 180.0310967558673 L 1000.8000000000001 181.2317761107381 L 996.0 181.9191903837738 L 991.1999999999999 183.08766613433684 L 986.4000000000001 183.59548492517865 L 981.5999999999999 185.57614493501677 L 976.7999999999998 187.09423020274946 L 972.0 188.42046366869448 L 967.1999999999999 189.6036540668411 L 962.4 190.93787522643996 L 957.6 192.45228707328215 L 952.8000000000001 193.644550083176 L 948.0 194.90112990706814 L 943.2 196.64583683259397 L 938.4000000000001 197.36530115524747 L 933.6 198.60951532423053 L 928.8 200.36946589144213 L 924.0 202.05755296698354 L 919.2 202.97730639339963 L 914.4000000000001 204.02049014456952 L 909.5999999999999 203.9934482435919 L 904.8 204.83726196540948 L 900.0 205.71081034118546 L 895.2 206.9244528030926 L 890.4000000000001 207.9196112590219 L 885.5999999999999 208.97317182139864 L 880.8 210.7586949630987 L 876.0 211.85109191329624 L 871.2 212.04260885749719 L 866.4 213.84949034298108 L 861.5999999999999 214.3659682635515 L 856.8 216.2933034620675 L 852.0 217.33450569515264 L 847.1999999999999 217.89456549460888 L 842.4 219.22179692321475 L 837.6 220.83290605416505 L 832.8000000000001 222.15971614590555 L 828.0 223.8676928362205 L 823.2 224.14062854827012 L 818.4000000000001 224.38574847156494 L 813.6 225.06664405079687 L 808.8 226.59817317195555 L 804.0 227.42021707864097 L 799.2 228.78039217450043 L 794.4000000000001 230.15718366462346 L 789.5999999999999 231.98522972104018 L 784.8 233.69351982649124 L 780.0 234.32790181005996 L 775.2 235.6114056106786 L 770.4000000000001 236.85646364017697 L 765.5999999999999 238.78744929808536 L 760.8 240.78773770846726 L 756.0 242.18573228506375 L 751.2 243.77896681099716 L 746.4 245.0446184563447 L 741.5999999999999 246.43569485028317 L 736.8 247.58002056761936 L 732.0 248.81014668560152 L 727.1999999999999 250.1589004829593 L 722.4 251.84381067486188 L 717.6 252.87618670537856 L 712.8000000000001 254.05211564418042 L 708.0 255.44449116709666 L 703.2 256.6606991266404 L 698.4000000000001 257.40784563569787 L 693.6 258.14160247491935 L 688.8 259.54343413690196 L 684.0 261.54713102881226 L 679.2 262.96487797249193 L 674.4000000000001 265.0646177056002 L 669.5999999999999 266.27741576035186 L 664.8 268.9989497669256 L 660.0 270.6719098965509 L 655.2 272.9138305269677 L 650.4000000000001 274.15746061378223 L 645.5999999999999 275.4660624633743 L 640.8 277.089808342762 L 636.0 278.22378424860483 L 631.2 279.44528016653936 L 626.4 280.48942982323456 L 621.5999999999999 282.33474542176634 L 616.8 284.53754383414895 L 612.0 285.933611689473 L 607.1999999999999 286.662513856375 L 602.4 287.29429424654086 L 597.6 288.8509862656574 L 592.8000000000001 289.75836259502285 L 588.0 291.21107275016055 L 583.2 292.4751505179837 L 578.4000000000001 294.2260013642348 L 573.6 295.63935756924604 L 568.8 296.735184016187 L 564.0 298.52676582087537 L 559.2 299.2509626397233 L 554.4000000000001 300.2719735962823 L 549.5999999999999 301.1083154394153 L 544.8 302.30884463357063 L 540.0 303.5546512427931 L 535.2 304.79551797487454 L 530.4000000000001 305.69818536100337 L 525.5999999999999 306.78857310115717 L 520.8 308.2817387361571 L 516.0 309.0745947466534 L 511.2 309.97727705992315 L 506.40000000000003 311.1223110928437 L 501.6 312.7752556250222 L 496.8 314.5116829545958 L 492.0 315.21704232673386 L 48

In [108]:
svg = toyplot.svg.render(vittaria, "git_data/figs/vittaria_medmut_exhet_fullsexcomp.svg")

In [109]:
vittaria = het_plot([neut_med_ob, pur_med_ob, pos_med_ob, lowsex_neut_med_ob, lowsex_pur_med_ob, lowsex_pos_med_ob, 
         medsex_neut_med_ob, medsex_pur_med_ob, medsex_pos_med_ob,
         highsex_neut_med_ob, highsex_pur_med_ob, highsex_pos_med_ob], 
         plot_error = False,
         ymax = 150,
         labels = ["Neutral", "Purifying", "Positive", "Low Sex, Neutral", "Low Sex, Purifying",
                  "Low Sex Postiive", "Med Sex Neutral", "Med Sex, Purifying", "Med Sex, Positive",
                  "High Sex, Neutral", "High Sex, Purifying", "High Sex, Positive"],
        title = "Observed Heterozygosity",
        palette = full)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600.0px" height="500.0px" viewBox="0 0 600.0 500.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t379cca86e6eb4a4d8ccc6427d44172c6"> <path d="M 1020.0 -60.855274970163634 L 1015.1999999999999 -57.86931128080013 L 1010.4000000000001 -54.38568697654277 L 1005.5999999999999 -52.41109808518951 L 1000.8000000000001 -50.773634126506195 L 996.0 -47.78767043714289 L 991.1999999999999 -46.08599220557009 L 986.4000000000001 -44.56090322444368 L 981.5999999999999 -41.462564557523464 L 976.7999999999998 -39.02242218772119 L 972.0 -37.256529683258876 L 967.1999999999999 -33.91738749300295 L 962.4 -31.47724512320059 L 957.6 -26.885924611598668 L 952.8000000000001 -26.01903192758988 L 948.0 -23.64310383067702 L 943.2 -20.769515118870345 L 938.4000000000001 -17.623015747282977 L 933.6 -14.749427035476145 L 928.8 -12.935373826346776 L 924.0 -9.532017363201504 L 919.2 -7.8784998362957595 L 914.4000000000001 -4.651732623596544 L 909.5999999999999 -5.1172861020456395 L 904.8 -2.5968758911314325 L 900.0 -0.46175131755431664 L 895.2 1.5288911420212798 L 890.4000000000001 4.065354921157919 L 885.5999999999999 4.675390513608562 L 880.8 8.09480054497645 L 876.0 11.128924939006986 L 871.2 11.8031748043472 L 866.4 15.383120517938806 L 861.5999999999999 16.779780953286206 L 856.8 20.247351689321235 L 852.0 22.896190446014614 L 847.1999999999999 22.831976173125028 L 842.4 25.866100567155662 L 837.6 29.494206985414486 L 832.8000000000001 32.287527856109385 L 828.0 34.43870599790886 L 823.2 35.14506299969369 L 818.4000000000001 37.087544754602234 L 813.6 38.53236589461679 L 808.8 41.90361522131744 L 804.0 44.680882523789904 L 799.2 46.94443564314602 L 794.4000000000001 50.70097060718393 L 789.5999999999999 53.3498093638774 L 784.8 57.25082644191666 L 780.0 59.11304035571325 L 775.2 61.50502202084855 L 770.4000000000001 64.71573566532537 L 765.5999999999999 67.12377089868298 L 760.8 71.0729486813895 L 756.0 75.51978707898994 L 751.2 77.62280451612223 L 746.4 79.4047505888068 L 741.5999999999999 83.09707127995526 L 736.8 86.17935637865291 L 732.0 88.41080236156436 L 727.1999999999999 89.72719495579996 L 722.4 93.29108710116921 L 717.6 95.42621167474618 L 712.8000000000001 98.2195325454411 L 708.0 100.96469271146881 L 703.2 103.18008512615779 L 698.4000000000001 106.63160229397042 L 693.6 107.17742361353142 L 688.8 109.21622677777425 L 684.0 113.48647592492846 L 679.2 116.55270745540383 L 674.4000000000001 121.38483149034153 L 669.5999999999999 122.57279553879783 L 664.8 127.74204450640553 L 660.0 130.71195462754662 L 655.2 134.5487574326965 L 650.4000000000001 135.7206679129305 L 645.5999999999999 140.6170062207577 L 640.8 142.92872004478096 L 636.0 145.2404338688043 L 631.2 148.2745582628349 L 626.4 148.77221887772885 L 621.5999999999999 152.28795031843094 L 616.8 155.91605673668977 L 612.0 158.74148474382946 L 607.1999999999999 160.29868086140073 L 602.4 161.95219838830627 L 597.6 166.52746533168573 L 592.8000000000001 167.42646515213931 L 588.0 171.72882143573824 L 583.2 172.724142665526 L 578.4000000000001 175.93485631000283 L 573.6 178.64790933958577 L 568.8 180.54223038982707 L 564.0 183.3837119651891 L 559.2 184.73221169586938 L 554.4000000000001 188.89008586546683 L 549.5999999999999 190.80046048393055 L 544.8 192.98374576217475 L 540.0 196.082084429095 L 535.2 197.65533411488863 L 530.4000000000001 199.9991550753567 L 525.5999999999999 201.52424405648313 L 520.8 204.0607078356199 L 516.0 206.91824297920425 L 511.2 208.12226059588312 L 506.40000000000003 210.49818869279596 L 501.6 213.43599167749227 L 496.8 217.0480445275287 L 492.0 216.42195536

In [110]:
svg = toyplot.svg.render(vittaria, "git_data/figs/vittaria_medmut_obhet_fullsexcomp.svg")

In [111]:
vittaria = het_plot([hom_neut_med_ex, hom_pur_med_ex, hom_pos_med_ex, 
                     neut_med_ex, pur_med_ex, pos_med_ex,
         lowsex_neut_med_ex, lowsex_pur_med_ex, lowsex_pos_med_ex, 
         medsex_neut_med_ex, medsex_pur_med_ex, medsex_pos_med_ex,
         highsex_neut_med_ex, highsex_pur_med_ex, highsex_pos_med_ex], 
         plot_error = False,
         ymax = 150,
         labels = ["Homosporous Neutral", "Homosporous Purifying", "Homosporous Positive",
                   "Neutral", "Purifying", "Positive", 
                  "Low Sex, Neutral", "Low Sex, Purifying",
                  "Low Sex Postiive", "Med Sex Neutral", "Med Sex, Purifying", "Med Sex, Positive",
                  "High Sex, Neutral", "High Sex, Purifying", "High Sex, Positive"],
        title = "Expected Heterozygosity - Control",
        palette = full2)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600.0px" height="500.0px" viewBox="0 0 600.0 500.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tfd0058237082404ea32fda1e29883701"> <path d="M 1015.2239999999999 411.0675639006805 L 1010.4240000000001 411.17286760186863 L 1005.624 411.73810784195575 L 1000.8240000000002 412.1943879456443 L 996.024 411.0471612666777 L 991.2239999999999 410.5997592527323 L 986.4240000000001 410.57767590617726 L 981.624 410.6199809849814 L 976.824 410.9351934884964 L 972.024 411.41711279797784 L 967.2239999999999 412.13487924097126 L 962.424 411.9134991737813 L 957.624 412.0925644919017 L 952.8240000000001 412.6541859458933 L 948.0239999999999 413.8893833337363 L 943.2239999999999 412.870868025354 L 938.424 413.2652967662776 L 933.624 413.96918495415343 L 928.8240000000001 413.2829427819914 L 924.0239999999999 412.2668227009175 L 919.2239999999999 411.51537748197694 L 914.424 411.7010234126211 L 909.624 411.4243128809736 L 904.824 411.34634699797425 L 900.024 410.1282487703863 L 895.224 409.5788851781382 L 890.4240000000001 409.012901213189 L 885.624 410.120040990825 L 880.8240000000001 410.9434728677504 L 876.024 410.73050452252403 L 871.224 411.42851304156824 L 866.424 412.12287533847126 L 861.624 412.5066490038123 L 856.8240000000001 412.0771415945907 L 852.024 412.4414596947767 L 847.2239999999999 412.41724641721925 L 842.424 412.6456011738732 L 837.624 412.76544264295535 L 832.8240000000001 413.37126435321386 L 828.0239999999999 413.2644720110627 L 823.2239999999999 412.40750312748344 L 818.424 412.200528779892 L 813.624 411.7929619207899 L 808.8240000000001 411.8885661355845 L 804.0239999999999 412.40833020376834 L 799.2239999999999 412.61184253890957 L 794.424 412.62452276913973 L 789.624 411.9531755188259 L 784.824 412.521054028039 L 780.024 412.6546682945213 L 775.224 413.0854761923559 L 770.4240000000001 412.43674453976985 L 765.624 411.93453190339426 L 760.8240000000001 411.1200530715714 L 756.024 411.0770240121299 L 751.224 411.44656742871365 L 746.424 410.4462774104833 L 741.624 411.2287017603372 L 736.8240000000001 411.9413965381951 L 732.024 412.0697781887999 L 727.2239999999999 411.4538232953565 L 722.424 412.412701511989 L 717.624 412.58523987664097 L 712.8240000000001 412.36166847928024 L 708.0239999999999 411.6747287425467 L 703.2239999999999 411.69366752689467 L 698.424 412.56051741672167 L 693.624 411.4698891888577 L 688.8240000000001 410.3665025449439 L 684.0239999999999 410.72765124120815 L 679.2239999999999 409.87229842469094 L 674.424 410.0778161904646 L 669.624 410.2721837061964 L 664.824 409.5012581930876 L 660.024 408.98499890328054 L 655.224 409.22686938540465 L 650.4240000000001 409.38662219331303 L 645.624 409.94196401307204 L 640.8240000000001 411.0158833953107 L 636.024 410.54023861863834 L 631.224 410.53622135591473 L 626.424 410.1896069513562 L 621.624 409.85067248260674 L 616.8240000000001 410.1034227356407 L 612.024 410.24695838378454 L 607.2239999999999 410.1618196406554 L 602.424 410.8125706458223 L 597.624 410.1061042415033 L 592.8240000000001 409.92870956849293 L 588.0239999999999 409.6590741781903 L 583.2239999999999 409.16487557872364 L 578.424 409.4728200557253 L 573.624 409.27245391820026 L 568.8240000000001 409.5482920384078 L 564.0239999999999 409.35907241316505 L 559.2239999999999 409.29534843448914 L 554.424 410.0363324684079 L 549.624 410.137296076285 L 544.824 410.44877070321604 L 540.024 410.49342451843944 L 535.2239999999999 409.9065220793671 L 530.424 410.48026660375723 L 525.624 409.9012346687648 L 520.8240000000001 409.7517266299239 L 516.0239999999999 408.82384156061005 L 511.224 409

In [112]:
svg = toyplot.svg.render(vittaria, "git_data/figs/vittaria_medmut_exhet_fullsexcomp_control.svg")

In [113]:
vittaria = het_plot([hom_neut_med_ob, hom_pur_med_ob, hom_pos_med_ob, 
                     neut_med_ob, pur_med_ob, pos_med_ob, 
         lowsex_neut_med_ob, lowsex_pur_med_ob, lowsex_pos_med_ob, 
         medsex_neut_med_ob, medsex_pur_med_ob, medsex_pos_med_ob,
         highsex_neut_med_ob, highsex_pur_med_ob, highsex_pos_med_ob], 
         plot_error = False,
         ymax = 150,
         labels = ["Homosporous Neutral", "Homosporous Purifying", "Homosporous Positive", 
                   "Neutral", "Purifying", "Positive", 
                  "Low Sex, Neutral", "Low Sex, Purifying",
                  "Low Sex Postiive", "Med Sex Neutral", "Med Sex, Purifying", "Med Sex, Positive",
                  "High Sex, Neutral", "High Sex, Purifying", "High Sex, Positive"],
        title = "Observed Heterozygosity - Control",
        palette = full2)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600.0px" height="500.0px" viewBox="0 0 600.0 500.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t6bab9ee3888f46899cb2185fd8e2d21f"> <path d="M 1015.2239999999999 410.92698794926224 L 1010.4240000000001 411.793880633271 L 1005.624 411.84204133793816 L 1000.8240000000002 411.05541649504136 L 996.024 409.85139887836255 L 991.2239999999999 410.6701308577042 L 986.4240000000001 410.413273766146 L 981.624 410.750398698816 L 976.824 410.39722019792356 L 972.024 411.1356843361533 L 967.2239999999999 412.05073772482916 L 962.424 411.6333449510472 L 957.624 412.09889842949633 L 952.8240000000001 413.1102732275065 L 948.0239999999999 412.9978982499498 L 943.2239999999999 413.2065946368408 L 938.424 411.9704698837172 L 933.624 414.7958978908569 L 928.8240000000001 412.9978982499498 L 924.0239999999999 412.11495199771866 L 919.2239999999999 411.15173790437564 L 914.424 410.36511306147884 L 909.624 411.50491640526815 L 904.824 410.4774880390355 L 900.024 410.5738094483698 L 895.224 409.3216311270238 L 890.4240000000001 408.93634548968663 L 885.624 410.3811666297012 L 880.8240000000001 411.08752363148614 L 876.024 411.0233093585965 L 871.224 411.53702354171287 L 866.424 412.19521983883067 L 861.624 412.211273407053 L 856.8240000000001 412.6768268855021 L 852.024 412.16311270238583 L 847.2239999999999 412.002577020162 L 842.424 412.1310055659411 L 837.624 412.56445190794545 L 832.8240000000001 413.43134459195426 L 828.0239999999999 414.55509436752106 L 823.2239999999999 412.9657911135051 L 818.424 411.66545208749193 L 813.624 411.6815056557144 L 808.8240000000001 411.50491640526815 L 804.0239999999999 411.32832715482186 L 799.2239999999999 412.80525543128124 L 794.424 411.7296663603815 L 789.624 411.7136127921591 L 784.824 412.48418406683356 L 780.024 412.06679129305155 L 775.224 412.9978982499498 L 770.4240000000001 412.66077331727973 L 765.624 411.56913067815765 L 760.8240000000001 411.0072557903742 L 756.024 410.63802372125934 L 751.224 412.5323447715007 L 746.424 410.2206309474774 L 741.624 411.0233093585965 L 736.8240000000001 412.06679129305155 L 732.024 412.09889842949633 L 727.2239999999999 411.15173790437564 L 722.424 411.5209699734905 L 717.624 412.9015768406155 L 712.8240000000001 412.75709472661407 L 708.0239999999999 410.6701308577042 L 703.2239999999999 410.62197015303695 L 698.424 412.5323447715007 L 693.624 412.54839833972306 L 688.8240000000001 410.3811666297012 L 684.0239999999999 410.12430953814305 L 679.2239999999999 409.4500596728029 L 674.424 409.7390239008058 L 669.624 410.62197015303695 L 664.824 410.06009526525355 L 660.024 408.63132769346134 L 655.224 409.4500596728029 L 650.4240000000001 408.9845061943538 L 645.624 409.4018989681358 L 640.8240000000001 411.4407021323786 L 636.024 411.1356843361533 L 631.224 410.9430415174847 L 626.424 410.42932733436834 L 621.624 410.26879165214456 L 616.8240000000001 409.8032381736954 L 612.024 410.62197015303695 L 607.2239999999999 410.3008987885893 L 602.424 410.87882724459513 L 597.624 410.60591658481457 L 592.8240000000001 408.8721312167971 L 588.0239999999999 410.3169523568117 L 583.2239999999999 409.4018989681358 L 578.424 409.5142739456925 L 573.624 409.9477202876968 L 568.8240000000001 408.7115955345733 L 564.0239999999999 408.53500628412706 L 559.2239999999999 408.0533992374555 L 554.424 410.65407728948173 L 549.624 408.67948839812846 L 544.824 410.0922024016983 L 540.024 409.91561315125205 L 535.2239999999999 409.48216680924776 L 530.424 410.6861844259265 L 525.624 409.08082760368814 L 520.8240000000001 409.3216311270238 L 516.0239999999999 409.2253097176895 L 511.

In [114]:
svg = toyplot.svg.render(vittaria, "git_data/figs/vittaria_medmut_obhet_fullsexcomp.svg")

In [115]:
vittaria = het_plot([neut_low_ex, pur_low_ex, pos_low_ex], 
         labels = ["Neutral", "Purifying", "Positive"],
         title = "Low Mutation Rate, Expected Heterozygosity, No Sex",
         palette=low,
         ymax = 15)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600.0px" height="500.0px" viewBox="0 0 600.0 500.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t434b01fa48694a1db462bb5b1e25b21e"> <polygon points="1020.0,-85.66500293714826 1015.1999999999999,-88.03839986952207 1010.4000000000001,-86.67836920781698 1005.5999999999999,-88.18365121751265 1000.8000000000001,-80.04009803545983 996.0,-79.05215985938811 991.1999999999999,-79.05598663745431 986.4000000000001,-76.11626824560639 981.5999999999999,-74.68335540548588 976.7999999999998,-72.12901814420341 972.0,-71.74381556674533 967.1999999999999,-69.52285242112657 962.4,-71.72545142484243 957.6,-65.3448640316334 952.8000000000001,-65.96475079241904 948.0,-65.53372933031035 943.2,-64.12395902097826 938.4000000000001,-61.35981765406578 933.6,-60.20322634715755 928.8,-57.858606449047386 924.0,-54.17002040471394 919.2,-57.86241822654634 914.4000000000001,-54.716118018649894 909.5999999999999,-46.798306143560794 904.8,-44.51520688223289 900.0,-45.626437714571026 895.2,-42.3989723017322 890.4000000000001,-40.387209074876594 885.5999999999999,-37.7236231247125 880.8,-31.238835995138174 876.0,-28.690332608624246 871.2,-22.584413938334137 866.4,-22.153428388750882 861.5999999999999,-18.982408830052492 856.8,-14.482441354671423 852.0,-10.83111857624506 847.1999999999999,-9.491130826625707 842.4,-5.133025632611378 837.6,-2.9508645189966956 832.8000000000001,-0.9186812186453182 828.0,-1.0990096660601978 823.2,2.2873238452558127 818.4000000000001,3.8559546021979116 813.6,4.494897931445763 808.8,3.147456515016742 804.0,3.7172590451981904 799.2,5.189618588633053 794.4000000000001,7.426902060183785 789.5999999999999,5.614846399526471 784.8,5.8264183344764575 780.0,2.9997909102972073 775.2,6.46033274297325 770.4000000000001,10.401124369044581 765.5999999999999,15.743232582477674 760.8,19.16066710708784 756.0,15.329568328224497 751.2,19.41823970820227 746.4,26.041905425255152 741.5999999999999,32.02748715242136 736.8,36.555765029906674 732.0,38.61021158769515 727.1999999999999,42.89591708495962 722.4,45.882764190362 717.6,48.81657903707287 712.8000000000001,46.349150412335106 708.0,55.23292440359787 703.2,61.93005262055136 698.4000000000001,67.59298382243055 693.6,67.49942295460164 688.8,76.84070619265219 684.0,78.52325015495161 679.2,81.86159195935332 674.4000000000001,85.34098218986824 669.5999999999999,89.27978089365148 664.8,97.12550108817551 660.0,98.48618490216413 655.2,103.71685041739437 650.4000000000001,106.31922010556524 645.5999999999999,103.02042834481182 640.8,104.15614073042346 636.0,105.79955650494679 631.2,110.44911771272822 626.4,107.38335600554265 621.5999999999999,112.7034958097477 616.8,115.72142168816877 612.0,116.05328937641289 607.1999999999999,118.36449276419754 602.4,118.10273837754548 597.6,127.44500641433706 592.8000000000001,133.28468810143036 588.0,137.22684900122243 583.2,141.9732375338129 578.4000000000001,151.12444568238925 573.6,152.23318957358603 568.8,151.54792875018939 564.0,155.83887107182903 559.2,156.29069621972457 554.4000000000001,149.01816908402216 549.5999999999999,152.24392769321275 544.8,156.83944221076212 540.0,157.51486041282948 535.2,160.42624187294683 530.4000000000001,159.31887573584868 525.5999999999999,158.15680835116243 520.8,161.13050723885985 516.0,165.56106482361957 511.2,167.5750247872592 506.40000000000003,174.05248496477503 501.6,176.428240755863 496.8,181.56479347029705 492.0,184.2835676938252 487.20000000000005,191.9648713691087 482.4,194.36243942431128 477.6,198.08093387210263 472.79999999999995,201.23637257091812 468.0,208.0815065129792 463.2,205.04609165159744 458.4,206.5042238484044

In [116]:
svg = toyplot.svg.render(vittaria, "../figs/vittaria_lowmut_exhet_nosex.svg")

In [119]:
vittaria = het_plot([neut_high_ex, pur_high_ex, pos_high_ex], 
         labels = ["Neutral", "Purifying", "Positive"],
         title = "High Mutation Rate, Expected Heterozygosity, No Sex",
         palette=high,
         ymax = 1500)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600.0px" height="500.0px" viewBox="0 0 600.0 500.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tda19b36b17254cd5af569694a7e06eda"> <polygon points="1020.0,-96.91381213334763 1015.1999999999999,-94.02146071199083 1010.4000000000001,-91.0081953989989 1005.5999999999999,-88.55235493174962 1000.8000000000001,-85.25796067554795 996.0,-82.83653187539338 991.1999999999999,-79.90964344610593 986.4000000000001,-77.7209083252325 981.5999999999999,-75.58744051670624 976.7999999999998,-73.42288457862863 972.0,-71.49361256696625 967.1999999999999,-68.976966827546 962.4,-66.95979895997817 957.6,-64.45562585515873 952.8000000000001,-62.36811559095159 948.0,-59.25580536179805 943.2,-56.6079689208391 938.4000000000001,-54.655737864172266 933.6,-52.72960356426883 928.8,-50.69107821532258 924.0,-48.271675070354206 919.2,-45.93423751836501 914.4000000000001,-43.42465791320792 909.5999999999999,-40.450933541785716 904.8,-37.65914883903504 900.0,-34.49758462597478 895.2,-31.78234388227513 890.4000000000001,-28.811465223209908 885.5999999999999,-25.92547093655554 880.8,-23.121431918402124 876.0,-20.777547667837993 871.2,-17.91068817627405 866.4,-15.109960411448824 861.5999999999999,-13.131039603278268 856.8,-10.73110209025036 852.0,-7.846399935348316 847.1999999999999,-5.079695853170989 842.4,-2.222737871291997 837.6,0.6689524498912931 832.8000000000001,2.3376446670395836 828.0,4.807299168352429 823.2,7.017201885275632 818.4000000000001,9.380081490491378 813.6,12.327477910211634 808.8,14.720339405636373 804.0,17.11719299577245 799.2,20.52170197143289 794.4000000000001,22.81897452381216 789.5999999999999,25.079387057097435 784.8,27.323773798522197 780.0,29.8437469070916 775.2,32.14317424804971 770.4000000000001,34.79871740618648 765.5999999999999,37.51976642987553 760.8,40.7631679822398 756.0,44.0042137487022 751.2,46.293695992030486 746.4,48.403248697578476 741.5999999999999,50.70140995269148 736.8,52.996631534194286 732.0,55.63886029500967 727.1999999999999,58.32162953700802 722.4,61.07434838652769 717.6,63.676182129545815 712.8000000000001,66.5706653680054 708.0,69.45495177892353 703.2,71.2973038021288 698.4000000000001,73.71562504099894 693.6,76.02494006947278 688.8,78.5301592952298 684.0,81.63214181165043 679.2,84.42003602451501 674.4000000000001,86.65850967937442 669.5999999999999,89.092237746579 664.8,91.54318954231077 660.0,94.23068551636774 655.2,96.81278094572815 650.4000000000001,99.08024355757541 645.5999999999999,101.77779270791166 640.8,104.34015604874789 636.0,106.99497228953373 631.2,110.0883084683631 626.4,112.73973911436073 621.5999999999999,115.20510262208873 616.8,118.21698361025162 612.0,120.82482111578054 607.1999999999999,123.55305361367485 602.4,126.45344822216396 597.6,129.00994255256384 592.8000000000001,131.21301891727714 588.0,133.0173405088795 583.2,135.8221857736408 578.4000000000001,138.54114729735068 573.6,140.80457483805932 568.8,142.86903270128508 564.0,145.07739193735438 559.2,147.3887724786536 554.4000000000001,149.52492513223297 549.5999999999999,152.03452438528294 544.8,153.88176786652343 540.0,156.48813581228256 535.2,159.3563820081972 530.4000000000001,162.49416922750876 525.5999999999999,165.40711299927818 520.8,167.98887613118598 516.0,170.03799664053182 511.2,173.10028225727854 506.40000000000003,175.38146818787928 501.6,177.2851789904293 496.8,179.89289859699022 492.0,182.54762438853683 487.20000000000005,185.54275870777326 482.4,187.8356447292592 477.6,190.98492032762167 472.79999999999995,193.525950032026 468.0,196.07111714519377 463.2,198.50512024197286 458.4,200.29053933482373 453.59999999999997

In [121]:
svg = toyplot.svg.render(vittaria, "git_data/figs/vittaria_highmut_exhet_nosex.svg")

### Ne Plots

#### Function for plotting the data

In [123]:
def Ne_plot(datasets, error=None, labels=[],
                        ymin = 0.0, 
                        ymax = 1000,
                        #xmin = 0, xmax = 0.3,
                        xlocations = [0, 4000, 8000, 12000, 16000, 20000],
                                     #24000, 28000, 32000, 36000, 40000], 
                        #ylocations=[0.0, 0.25, 0.50, 0.75, 0.9], #1.0
                        palette = "Set2",
                        plot_error = True,
                        title = "Ne",):
    xlabel = "Time (generations)"
    title = title
    
    #conf = int(100 - float(CIs[0][4:8]))

    #make the colors
    if isinstance(palette, str):
        palette = toyplot.color.brewer.palette(palette) #count=len(dataset)

    if error=="CI":
        error_str = "CI=95%"
    elif error=="std":
        error_str= "(σ)"
    
    label_style = {"text-anchor":"start", "-toyplot-anchor-shift":"5px", "font-size":"14px"}
    canvas = toyplot.Canvas(width=600, height=500)
    axes = canvas.cartesian(label = title, 
                            xlabel=xlabel, 
                            ylabel= "Ne",
                            ymin = ymin, 
                            ymax = ymax,
                            #xmin = xmin, 
                            xmax = 20000,
                           )

    xlocations=xlocations
    #ylocations=ylocations
    axes.x.ticks.locator = toyplot.locator.Explicit(locations=xlocations) #format="{:.2f}"
    axes.x.ticks.labels.style = {"font-size":"13px"}
    axes.y.ticks.labels.style = {"font-size":"13px"}
    #axes.y.ticks.locator = toyplot.locator.Explicit(locations=ylocations)
    #axes.x.ticks.locator = toyplot.locator.Uniform(count=6)

    for x in range(len(datasets)):
        data = datasets[x]
        time = data['time'].tolist()

        Ne_str = data['Ne'].tolist()
        Ne=[float(i) for i in Ne_str]

        #format the error
        if error == "CI":
            for column in data.columns:
                if "CI_95%" in column:
                    error_plus = column
                if "CI_5%" in column:
                    error_minus = column
            CI_plus_str = data[error_plus].tolist()
            CI_plus=[float(i) for i in CI_plus_str]

            a = [mean_heterozygosity, CI_plus]
            y1 = list(map(sum,zip(*a)))

            CI_minus_str = data[error_minus].tolist()
            CI_minus=[float(i) for i in CI_minus_str]
            minus_CI = []
            for i in CI_minus:
                minus_CI.append(i*-1)

            b = [mean_heterozygosity, minus_CI]
            y2 = list(map(sum,zip(*b)))

        elif error == "std":
            prob_std_str = data['stdev'].tolist()
            prob_std=[float(i) for i in prob_std_str]

            a = [mean_heterozygosity, prob_std]
            y1 = list(map(sum,zip(*a)))

            minus_prob_std = []
            for i in prob_std:
                minus_prob_std.append(i*-1)

            b = [mean_heterozygosity, minus_prob_std]
            y2 = list(map(sum,zip(*b)))

        if plot_error == True:
            mark = axes.fill(time, y1, y2, opacity=0.2, color = palette[x])
        mark = axes.plot(time, Ne, color=palette[x])
        axes.text(16000, Ne[110], f"{labels[x]}", style=label_style, color = palette[x])

    #mark = axes.vlines(x=0.0)
    set_axes_ticks_external(axes)
    axes.label.style = {"font-size":"19px"}
    axes.x.label.style = {"font-size":"16px"}
    axes.y.label.style = {"font-size":"16px"}
    #set_axes_box_outline(axes)

    return canvas

In [124]:
vittaria = Ne_plot([hom_neut_med_theta, hom_pur_med_theta, hom_pos_med_theta, 
                    neut_med_theta, pur_med_theta, pos_med_theta, lowsex_neut_med_theta, 
                    lowsex_pur_med_theta, lowsex_pos_med_theta, 
                    medsex_neut_med_theta, medsex_pur_med_theta, medsex_pos_med_theta,
                    highsex_neut_med_theta, highsex_pur_med_theta, highsex_pos_med_theta], 
                    plot_error = False,
                    ymax = 3000,
         labels = ["Homosporous Neutral", "Homosporous Purifying", "Homosporous Positive",
                   "Neutral", "Purifying", "Positive", "Low Sex, Neutral", "Low Sex, Purifying",
                  "Low Sex Postiive", "Med Sex Neutral", "Med Sex, Purifying", "Med Sex, Positive",
                  "High Sex, Neutral", "High Sex, Purifying", "High Sex, Positive"],
        title = "Effective Population Size",
        palette = full2)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600.0px" height="500.0px" viewBox="0 0 600.0 500.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tabb6dcaacb9345a1aae0d8d18e586b99"> <path d="M 1015.2239999999999 287.72496457594906 L 1010.4240000000001 289.4186033005281 L 1005.624 290.3461559242264 L 1000.8240000000002 294.14496050455654 L 996.024 299.13780884218863 L 991.2239999999999 290.4353084679468 L 986.4240000000001 292.3639822513526 L 981.624 292.1792583218036 L 976.824 288.2428200447688 L 972.024 291.7997073855821 L 967.2239999999999 296.0065371743537 L 962.424 297.969170199235 L 957.624 296.7009937799552 L 952.8240000000001 294.9392676929262 L 948.0239999999999 296.49688936347 L 943.2239999999999 291.7579857658764 L 938.424 295.9069864904443 L 933.624 293.7514190842321 L 928.8240000000001 293.41531510620774 L 924.0239999999999 294.2377116069617 L 919.2239999999999 295.4908153611004 L 914.424 296.2080543814177 L 909.624 295.4952070591405 L 904.824 294.6318281832817 L 900.024 295.73596411747246 L 895.224 293.7442762885225 L 890.4240000000001 286.3112583755909 L 885.624 287.3606285602299 L 880.8240000000001 295.493813856003 L 876.024 291.76771837898053 L 871.224 296.1757741602679 L 866.424 296.4884605038368 L 861.624 299.09843668851437 L 856.8240000000001 295.7193837092908 L 852.024 295.28999500498287 L 847.2239999999999 298.081682199094 L 842.424 294.55269381368805 L 837.624 296.5006108156335 L 832.8240000000001 295.1445920737931 L 828.0239999999999 294.61328723150103 L 823.2239999999999 292.86963948954525 L 818.424 299.8466969757256 L 813.624 296.46947135089613 L 808.8240000000001 299.13897876420947 L 804.0239999999999 293.85299346145644 L 799.2239999999999 301.0055410942096 L 794.424 298.885040709103 L 789.624 292.76311831379877 L 784.824 294.51556524787026 L 780.024 299.534330607954 L 775.224 299.75762345536685 L 770.4240000000001 294.35923718701184 L 765.624 294.3485555636543 L 760.8240000000001 292.3218910351676 L 756.024 292.6819776928013 L 751.224 293.57655496767615 L 746.424 295.0057853964695 L 741.624 297.6149775861484 L 736.8240000000001 294.7266059493546 L 732.024 295.92750419425283 L 727.2239999999999 296.7664411923935 L 722.424 296.78785649896 L 717.624 293.7335612272606 L 712.8240000000001 294.80057669527076 L 708.0239999999999 292.1877784998652 L 703.2239999999999 294.95163227521755 L 698.424 294.84617356809633 L 693.624 296.72349437615327 L 688.8240000000001 296.1053545558746 L 684.0239999999999 290.00833088949963 L 679.2239999999999 293.74995592783665 L 674.424 288.6741228521558 L 669.624 289.73475230326227 L 664.824 281.0854407969677 L 660.024 282.2799978699813 L 655.224 287.1032829599853 L 650.4240000000001 289.94123929563335 L 645.624 288.43089473370327 L 640.8240000000001 290.8947407476093 L 636.024 291.90116308012733 L 631.224 289.31557138462694 L 626.424 289.3168755926375 L 621.624 289.58714782625265 L 616.8240000000001 289.43097389026974 L 612.024 287.94504308117615 L 607.2239999999999 286.85555522599276 L 602.424 292.7755346246205 L 597.624 290.6335970330373 L 592.8240000000001 297.1538866488626 L 588.0239999999999 288.6643028774993 L 583.2239999999999 293.5521384422828 L 578.424 295.41724114485913 L 573.624 294.65304879789323 L 568.8240000000001 294.00830288455677 L 564.0239999999999 296.69274004758324 L 559.2239999999999 288.37561858490545 L 554.424 292.60764809157644 L 549.624 288.65078863212545 L 544.824 292.1949538589552 L 540.024 292.1455361835761 L 535.2239999999999 292.07842847068616 L 530.424 287.7847562246114 L 525.624 292.0983202130536 L 520.8240000000001 290.9762617594967 L 516.0239999999999 292.16013634896274 L 511.224 284.0

In [128]:
svg = toyplot.svg.render(vittaria, "git_data/figs/vittaria_Ne_control.svg")

### Fis Plots

In [125]:
def Fis_plot(datasets, error=None, labels=[],
                        ymin = -1.0, 
                        ymax = 1.0,
                        #xmin = 0, xmax = 0.3,
                        xlocations = [0, 4000, 8000, 12000, 16000, 20000],
                                     #24000, 28000, 32000, 36000, 40000], 
                        #ylocations=[0.0, 0.25, 0.50, 0.75, 0.9], #1.0
                        palette = "Set2",
                        plot_error = True,
                        title = "Ne",):
    xlabel = "Time (generations)"
    title = title
    
    #conf = int(100 - float(CIs[0][4:8]))

    #make the colors
    if isinstance(palette, str):
        palette = toyplot.color.brewer.palette(palette) #count=len(dataset)

    if error=="CI":
        error_str = "CI=95%"
    elif error=="std":
        error_str= "(σ)"
    
    label_style = {"text-anchor":"start", "-toyplot-anchor-shift":"5px", "font-size":"14px"}
    canvas = toyplot.Canvas(width=600, height=500)
    axes = canvas.cartesian(label = title, 
                            xlabel=xlabel, 
                            ylabel= "Ne",
                            ymin = ymin, 
                            ymax = ymax,
                            #xmin = xmin, 
                            xmax = 20000,
                           )

    xlocations=xlocations
    #ylocations=ylocations
    axes.x.ticks.locator = toyplot.locator.Explicit(locations=xlocations) #format="{:.2f}"
    axes.x.ticks.labels.style = {"font-size":"13px"}
    axes.y.ticks.labels.style = {"font-size":"13px"}
    #axes.y.ticks.locator = toyplot.locator.Explicit(locations=ylocations)
    #axes.x.ticks.locator = toyplot.locator.Uniform(count=6)

    for x in range(len(datasets)):
        data = datasets[x]
        time = data['time'].tolist()

        Fis_str = data['Fis'].tolist()
        Fis=[float(i) for i in Fis_str]

        #format the error
        if error == "CI":
            for column in data.columns:
                if "CI_95%" in column:
                    error_plus = column
                if "CI_5%" in column:
                    error_minus = column
            CI_plus_str = data[error_plus].tolist()
            CI_plus=[float(i) for i in CI_plus_str]

            a = [mean_heterozygosity, CI_plus]
            y1 = list(map(sum,zip(*a)))

            CI_minus_str = data[error_minus].tolist()
            CI_minus=[float(i) for i in CI_minus_str]
            minus_CI = []
            for i in CI_minus:
                minus_CI.append(i*-1)

            b = [mean_heterozygosity, minus_CI]
            y2 = list(map(sum,zip(*b)))

        elif error == "std":
            prob_std_str = data['stdev'].tolist()
            prob_std=[float(i) for i in prob_std_str]

            a = [mean_heterozygosity, prob_std]
            y1 = list(map(sum,zip(*a)))

            minus_prob_std = []
            for i in prob_std:
                minus_prob_std.append(i*-1)

            b = [mean_heterozygosity, minus_prob_std]
            y2 = list(map(sum,zip(*b)))

        if plot_error == True:
            mark = axes.fill(time, y1, y2, opacity=0.2, color = palette[x])
        mark = axes.plot(time, Fis, color=palette[x])
        axes.text(16000, Fis[110], f"{labels[x]}", style=label_style, color = palette[x])

    #mark = axes.vlines(x=0.0)
    set_axes_ticks_external(axes)
    axes.label.style = {"font-size":"19px"}
    axes.x.label.style = {"font-size":"16px"}
    axes.y.label.style = {"font-size":"16px"}
    #set_axes_box_outline(axes)

    return canvas

In [126]:
vittaria = Fis_plot([hom_neut_med_theta, hom_pur_med_theta, hom_pos_med_theta,
                    neut_med_theta, pur_med_theta, pos_med_theta, lowsex_neut_med_theta, 
                    lowsex_pur_med_theta, lowsex_pos_med_theta, 
                    medsex_neut_med_theta, medsex_pur_med_theta, medsex_pos_med_theta,
                    highsex_neut_med_theta, highsex_pur_med_theta, highsex_pos_med_theta], 
                    plot_error = False,
         labels = ["Homosporous Neutral", "Homosporous Purifying", "Homosporous Positive", 
                  "Neutral", "Purifying", "Positive", "Low Sex, Neutral", "Low Sex, Purifying",
                  "Low Sex Postiive", "Med Sex Neutral", "Med Sex, Purifying", "Med Sex, Positive",
                  "High Sex, Neutral", "High Sex, Purifying", "High Sex, Positive"],
        title = "Effective Population Size",
        palette = full2)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600.0px" height="500.0px" viewBox="0 0 600.0 500.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tc72af471775643a596c51d0dc6d59f9c"> <path d="M 1015.2239999999999 250.9231656358895 L 1010.4240000000001 245.90689513140404 L 1005.624 249.30127239442325 L 1000.8240000000002 257.7827661261877 L 996.024 257.84706659932397 L 991.2239999999999 249.54522124292893 L 986.4240000000001 251.06165666948473 L 981.624 249.15659123243185 L 976.824 253.5167935891259 L 972.024 251.87074900337194 L 967.2239999999999 250.5737239829407 L 962.424 251.8951916669444 L 957.624 249.95687715044173 L 952.8240000000001 246.83108415295084 L 948.0239999999999 256.4870994080459 L 943.2239999999999 247.64872476413518 L 938.424 259.2021634029639 L 933.624 243.96578794413472 L 928.8240000000001 252.02711176781102 L 924.0239999999999 251.04046620034129 L 919.2239999999999 252.4255725944947 L 914.424 258.9693337825595 L 909.624 249.4640664438911 L 904.824 255.76133179900967 L 900.024 247.1660004742893 L 895.224 251.60672184428307 L 890.4240000000001 250.469407851491 L 885.624 248.33956025939509 L 880.8240000000001 249.05805518377304 L 876.024 248.0992867165548 L 871.224 249.2784066486469 L 866.424 249.5069270868078 L 861.624 252.04127039268678 L 856.8240000000001 245.91946484780053 L 852.024 251.91903954159127 L 847.2239999999999 252.85639304300852 L 842.424 253.57431234839368 L 837.624 251.40219791903482 L 832.8240000000001 249.57131853678342 L 828.0239999999999 240.82800056058113 L 823.2239999999999 246.15566804866555 L 818.424 253.65706853742128 L 813.624 250.75075909441048 L 808.8240000000001 252.59301781302506 L 804.0239999999999 257.43704824011206 L 799.2239999999999 248.6582357866601 L 794.424 256.210767258238 L 789.624 251.6228902525931 L 784.824 250.2549330908177 L 780.024 254.0846690573102 L 775.224 250.61824645964677 L 770.4240000000001 248.4557169675293 L 765.624 252.47372438457097 L 760.8240000000001 250.74208874000337 L 756.024 252.8838683578197 L 751.224 242.77502995079513 L 746.424 251.45067437245052 L 741.624 251.3563710613837 L 736.8240000000001 249.15088420365726 L 732.024 249.80190469772404 L 727.2239999999999 252.01064488881343 L 722.424 256.1415579487908 L 717.624 247.8076035360257 L 712.8240000000001 247.28163811418557 L 708.0239999999999 256.7386326642808 L 703.2239999999999 257.1935317380459 L 698.424 250.1950766591801 L 693.624 242.8175274179311 L 688.8240000000001 249.9059788299336 L 684.0239999999999 253.91615051210863 L 679.2239999999999 252.66284378375803 L 674.424 252.15126460838874 L 669.624 247.7644027316744 L 664.824 246.51857626180995 L 660.024 252.16661381555645 L 655.224 248.62197463310545 L 650.4240000000001 252.49570760484875 L 645.624 253.4138078277142 L 640.8240000000001 247.21517957080528 L 636.024 246.15968762057213 L 631.224 247.37658120392524 L 626.424 248.4721143160377 L 621.624 247.3650277218837 L 616.8240000000001 251.9077457016327 L 612.024 247.60521169307503 L 607.2239999999999 249.11438841815394 L 602.424 249.5686926171512 L 597.624 246.82328639946007 L 592.8240000000001 256.6757988746592 L 588.0239999999999 245.8802557749172 L 583.2239999999999 248.5395082772155 L 578.424 249.74199257487487 L 573.624 245.82457349953035 L 568.8240000000001 255.220473542209 L 564.0239999999999 255.1099159473388 L 559.2239999999999 257.68516610367425 L 554.424 246.08287216909324 L 549.624 259.2752303862804 L 544.824 252.2925603739859 L 540.024 253.72066761302779 L 535.2239999999999 252.67923506665903 L 530.424 248.67463619379612 L 525.624 255.1788616791804 L 520.8240000000001 252.70158050184276 L 516.0239999999999 247.5532922

In [127]:
svg = toyplot.svg.render(vittaria, "git_data/figs/vittaria_Fis_control.svg")